## Ingeniería de Software III - Actualidad Informática

### Ciencia de datos (demo)

#### DS02 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../data/datos_creditos.csv", sep=";")
display(df_creditos.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [3]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ['id_cliente'] # Para generar resultados anónimos

# Se ejecuta la operación

df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)

In [4]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

Vista del dataset de datos de créditos:


,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


Limpieza de los datos (filtrado a nivel de filas)

In [5]:
# Se obtienen las cantidades de valores nulos por columna

nulos_x_columna = df_creditos.isna().sum()

print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna}")

Cantidad de filas que tienen valores nulos por atributo:
edad                     0
importe_solicitado       0
duracion_credito         0
antiguedad_empleado    337
situacion_vivienda       0
ingresos                 0
objetivo_credito         0
pct_ingreso              0
tasa_interes           912
estado_credito           0
falta_pago               0
dtype: int64


En función de este resultado se podría tomar alguna decisión extra.

In [6]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

df_temp = df_creditos[df_creditos['edad'] < 90]

print(f"Filas después del filtro: {df_temp.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 10123


Integración de datos:
* No aplica en este caso al no haber otros datos con los que hacer la integración

#### Transformación de atributos

Atributos nominales que se modifican los valores

In [7]:
# Columna: estado_credito
cambios_estado_credito = {
    0: 'P',
    1 : 'C',
}

estado_credito_N = df_temp.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')

Atributos numéricos que se discretizan

In [8]:
# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_temp['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

display(edad_N.value_counts())

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_temp['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

display(pct_ingreso_N.value_counts())

# Antiguedad del empleado

etiquetas_a_e = ['menor_5', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
valor_para_nan = 'NA'
antiguedad_empleados_N = pd.cut(df_temp['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

display(antiguedad_empleados_N.value_counts())

edad
menor_25    7149
25_a_30     2974
Name: count, dtype: int64

pct_ingreso
hasta_20    6445
20_a_40     3191
40_a_60      464
mayor_60      23
Name: count, dtype: int64

antiguedad_empleado
menor_5     4815
5_a_10      4705
NA           339
mayor_10     264
Name: count, dtype: int64

In [9]:
df_temp.columns

Index(['edad', 'importe_solicitado', 'duracion_credito', 'antiguedad_empleado',
       'situacion_vivienda', 'ingresos', 'objetivo_credito', 'pct_ingreso',
       'tasa_interes', 'estado_credito', 'falta_pago'],
      dtype='object')

In [10]:
# Agregar en este listado otros atributos que pudieran discretizarse o transformarse
col_eliminar_final = ['edad', 'pct_ingreso', 'antiguedad_empleado']

df_temp.drop(col_eliminar_final, inplace=True, axis=1)

df_final = pd.concat([pct_ingreso_N,  
                      edad_N,  
                      estado_credito_N,
                      antiguedad_empleados_N,
                      df_temp], axis=1)
df_final.head(5)


/tmp/ipykernel_26106/2519742947.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.drop(col_eliminar_final, inplace=True, axis=1)


,pct_ingreso,edad,estado_credito_N,antiguedad_empleado,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,tasa_interes,estado_credito,falta_pago
0,40_a_60,menor_25,C,NA,35000,3,ALQUILER,59000,PERSONAL,16.02,1,Y
1,hasta_20,menor_25,P,5_a_10,1000,2,PROPIA,9600,EDUCACIÓN,11.14,0,N
2,40_a_60,25_a_30,C,menor_5,5500,3,HIPOTECA,9600,SALUD,12.87,1,N
3,40_a_60,menor_25,C,5_a_10,35000,2,ALQUILER,65500,SALUD,15.23,1,N
4,40_a_60,menor_25,C,5_a_10,35000,4,ALQUILER,54400,SALUD,14.27,1,Y


Exportación de metadatos

In [18]:
import dtale as dt

informe = dt.show(df_final)
informe.open_browser()

In [11]:
print(f"Cantidad de columnas del dataset final: {df_final.shape[1]}")
print(f"Cantidad de filas del dataset final: {df_final.shape[0]}")

Cantidad de columnas del dataset final: 12
Cantidad de filas del dataset final: 10123


Exportación del dataset

In [12]:
df_final.to_csv("../data/datos_finales.csv", sep=';', index=False)